## Download and create model.onnx

Switch to the right Kernel - `yolov8n`

It was created as follows

```
conda create -n "yolov8n" python=3.11.7
conda init
#New terminal
conda activate yolov8n
cd /opt/conda/envs/yolov8n/bin
./pip install ipykernel
./python3 -m ipykernel install --user --name=yolov8n
./pip install "numpy<2.0" ultralytics onnx onnxruntime
```

In [ ]:
import os

In [ ]:
output_folder = "/mnt/models"
export_model_name= "yolov8n_dev"

In [ ]:
from pathlib import Path
import shutil

import torch
from torch.serialization import add_safe_globals

# Allowlist the Ultralytics DetectionModel class for safe unpickling
from ultralytics.nn.tasks import DetectionModel
add_safe_globals([DetectionModel])
from ultralytics import YOLO
# ------------------------------------------------------------------
# Settings
# ------------------------------------------------------------------
model_name = "/tmp/yolov8n.pt"                # or yolov8s.pt, etc.
export_size = 640                        # imgsz
output_dir = Path(f"{output_folder}/{export_model_name}")    # Triton version directory
output_dir.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# Load pretrained YOLO model (now allowed by safe loader)
# ------------------------------------------------------------------
model = YOLO(model_name)

# ------------------------------------------------------------------
# Export to ONNX
#   - Raw head, no NMS
#   - Output: [1, 84, N]  (N depends on strides, often 8400)
# ------------------------------------------------------------------
onnx_file = model.export(
    format="onnx",
    imgsz=export_size,
    opset=13,
    simplify=False,
    dynamic=False,
)
'''
dynamic=False produces an output with the shape below
[batch = 1, channels = 3, height = 640, width = 640]

YOLOv8 raw head produces:[1, 84, 8400]

- 84 = 4 box coords + 80 class logits

- 8400 = number of grid anchors at 640×640

- batch dimension is fixed at 1

"No dynamic dimensions" means:

The ONNX graph will not contain:

- "batch_size" variables
- symbolic dims like "height" or "width"
- ops like Resize or Pad that depend on variable shapes
- anything that depends on runtime-resolved dimension values

Everything is hard wired.

If Triton sends an image of shape [1, 3, 640, 640], it works.
If Triton sends [1, 3, 720, 720], it will not load — the graph cannot accept it.

'''


onnx_file = Path(onnx_file)

# Move into Triton repo as model.onnx
target_path = output_dir / "model.onnx"
shutil.move(str(onnx_file), str(target_path))

print(f"Exported ONNX written to: {target_path}")


In [ ]:
def write_config(model_name: str, folder: str):
    config = f'''
name: "{model_name}"
platform: "onnxruntime_onnx"
max_batch_size: 0

input: [
  {{ name: "images" data_type: TYPE_FP32 dims: [1, 3, 640, 640] }}
]
output: [
  {{ name: "output0" data_type: TYPE_FP32 dims: [1, 84, 8400] }}
]
instance_group: [ {{ kind: KIND_GPU, count: 1 }} ]

version_policy {{
  all {{}}
}}

# Only load 3 most recent versions
#version_policy {{
#  latest {{
#    num_versions: 3
#  }}
#}}

# Load only specific versions
#version_policy {{
#  specific {{
#    versions: [1, 3, 5]
#  }}
#}}
'''.strip()

    # Ensure folder exists
    os.makedirs(folder, exist_ok=True)

    # Write file
    path = os.path.join(folder, "config.pbtxt")
    with open(path, "w") as f:
        f.write(config)

    return path

In [ ]:
out_path = write_config(export_model_name, f"{output_folder}/{export_model_name}")
print("Wrote:", out_path)